In [1]:
# Import Chess AI
import ai
import evaluation
import chess

# Import frontend & backend libraries
import time
import chess.svg
import chess.pgn
import chess.engine
from IPython.display import SVG
import traceback
from flask import Flask, Response, request
import webbrowser

In [3]:
# This section is adapted from https://medium.com/dscvitpune/lets-create-a-chess-ai-8542a12afef

# Searching Ai's Move
def aimove():
    move = ai.determine_move(3, board)
    board.push(move)
    
# Searching Stockfish's Move
def stockfish():
    engine = chess.engine.SimpleEngine.popen_uci(
        "./stockfish")
    move = engine.play(board, chess.engine.Limit(time=0.1))
    board.push(move.move)

app = Flask(__name__)
# Front Page of the Flask Web Page
@app.route("/")
def main():
    global count, board
    ret = '<html><head>'
    ret += '<style>input {font-size: 20px; } button { font-size: 20px; }</style>'
    ret += '</head><body>'
    ret += '<img width=510 height=510 src="/board.svg?%f"></img></br></br>' % time.time()
    ret += '<form action="/game/" method="post"><button name="New Game" type="submit">New Game</button></form>'
    ret += '<form action="/undo/" method="post"><button name="Undo" type="submit">Undo Last Move</button></form>'
    ret += '<form action="/move/"><input type="submit" value="Make Human Move:"><input name="move" type="text"></input></form>'
    ret += '<form action="/dev/" method="post"><button name="Comp Move" type="submit">Make Ai Move</button></form>'
    ret += '<form action="/engine/" method="post"><button name="Stockfish Move" type="submit">Make Stockfish Move</button></form>'
    return ret
# Display Board
@app.route("/board.svg/")
def board():
    return Response(chess.svg.board(board=board, size=700), mimetype='image/svg+xml')
# Human Move
@app.route("/move/")
def move():
    try:
        move = request.args.get('move', default="")
        board.push_san(move)
    except Exception:
        traceback.print_exc()
    return main()
# Make Ai’s Move
@app.route("/dev/", methods=['POST'])
def dev():
    try:
        aimove()
    except Exception:
        traceback.print_exc()
    return main()
# Make UCI Compatible engine's move
@app.route("/engine/", methods=['POST'])
def engine():
    try:
        stockfish()
    except Exception:
        traceback.print_exc()
    return main()
# New Game
@app.route("/game/", methods=['POST'])
def game():
    board.reset()
    return main()
# Undo
@app.route("/undo/", methods=['POST'])
def undo():
    try:
        board.pop()
    except Exception:
        traceback.print_exc()
    return main()

In [ ]:
board = chess.Board()
webbrowser.open("http://127.0.0.1:5000/")
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2021 17:22:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:22:53] "GET /board.svg?1624396973.061449 HTTP/1.1" 308 -
127.0.0.1 - - [22/Jun/2021 17:22:53] "GET /board.svg/?1624396973.061449 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:23:06] "POST /dev/ HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:23:07] "GET /board.svg?1624396986.983219 HTTP/1.1" 308 -
127.0.0.1 - - [22/Jun/2021 17:23:07] "GET /board.svg/?1624396986.983219 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:23:16] "GET /move/?move=g7g5 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:23:16] "GET /board.svg?1624396996.020833 HTTP/1.1" 308 -
127.0.0.1 - - [22/Jun/2021 17:23:16] "GET /board.svg/?1624396996.020833 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:23:18] "POST /dev/ HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 17:23:18] "GET /board.svg?1624396998.284426 HTTP/1.1" 308 -
127.0.0.1 - - [22/Jun/2021 17:23:18] "GET /board.svg/?162439699

## Simulation against Stockfish

In [2]:
# Set up backend and stockfish engine
count = 0
movehistory = []
game = chess.pgn.Game()
board = chess.Board()
engine = chess.engine.SimpleEngine.popen_uci("./stockfish")
while not board.is_game_over(claim_draw=True):
    if board.turn:
        count += 1
        move = engine.play(board, chess.engine.Limit(time=0.1))
        movehistory.append(move.move)
        board.push(move.move)
    else:
        move = ai.determine_move(3, board)
        movehistory.append(move)
        board.push(move)
        game.add_line(movehistory)
game.headers["Result"] = str(board.result(claim_draw=True))
SVG(chess.svg.board(board=board, size=400))
print("Lasted {} turns".format(count))

AttributeError: module 'chess' has no attribute 'pgn'